In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import io #Used as buffer

In [2]:
path= Path(f'../../data/cluster_results')

In [3]:
def read_csv(f):
    df = pd.read_csv(f).mean()
    df['experiment'] = f.parent.parent.parent.name
    df['function'] = f.parent.parent.name
    name = f.parent.name
    df['freq'] = int(name[4:].split('n')[0])
    if 'noNN' in f.name:
        df['method'] = f.name.split('_')[0]       
    else:
        df['method'] = f.name.split('_')[0]+'_'+f.name.split('_')[1]
    return df

measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0

In [9]:
m = measure_name[index_measure]
files  = list(path.glob(f'exp*/*/freq1*nn_s2nn_tw5*bs4*/*{m}.csv'))
# files += list(path.glob(f'exp*/*/*nn_p3/*{m}.csv'))
files += [f for f in path.glob(f'exp*/*/freq1*nn_p3/*{m}.csv') if 'noNN' in f.name]
#print(files)
df = pd.DataFrame([read_csv(f) for f in files])
# data = pd.pivot_table(df, index=['experiment','function'], columns='method', values=m) #,'freq' later for adding frequency experiment
data = pd.pivot_table(df, index=['experiment', 'freq','function'],  columns='method') #,'freq' later for adding frequency experiment, ,values=m
data.style.format("{:,.2f}")

In [10]:
# with std
def read_csv(f):
    df = pd.read_csv(f).mean()
    df['std'] = pd.read_csv(f).std().iloc[0]
    df['experiment'] = f.parent.parent.parent.name
    df['function'] = f.parent.parent.name
    name = f.parent.name
    df['freq'] = int(name[4:].split('n')[0])
    if 'noNN' in f.name:
        df['method'] = f.name.split('_')[0]       
    else:
        df['method'] = f.name.split('_')[0]+'_'+f.name.split('_')[1]
    return df

measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0

In [18]:
m = measure_name[index_measure]
h=['noNN','Closest']
files  = list(f for f in path.glob(f'exp*/*/freq1*nn_s2nn_tw5*bs4*/*{m}.csv') if 'Closest' not in f.name and 'Random' not in f.name) #'noNN' not in f.name and
# files += list(path.glob(f'exp*/*/*nn_p3/*{m}.csv'))
#files += [f for f in path.glob(f'exp*/*/freq1*nn_p3/*{m}.csv') if 'noNN' in f.name]
#print(files)
df = pd.DataFrame([read_csv(f) for f in files])
# data = pd.pivot_table(df, index=['experiment','function'], columns='method', values=m) #,'freq' later for adding frequency experiment
data = pd.pivot_table(df, index=[ 'experiment','function','freq'],  columns='method') #,'freq' later for adding frequency experiment, ,values=m
data.style.format("{:,.2f}")

In [19]:
final_data = data[m].copy()

for col in data[m].columns:
    final_data[col] = data[m][col].apply(lambda x: f'{x:,.2f} (±') + data['std'][col].apply(lambda x: f'{x:,.2f})')

In [20]:
final_data

method                                       NNdrop_Worst  \
experiment function   freq                                  
exp1       rastrigin  1                     20.96 (±4.40)   
           rosenbrock 1          201,409.54 (±165,798.33)   
           sphere     1                     10.86 (±7.31)   
exp2       rastrigin  1                      1.90 (±0.22)   
           rosenbrock 1                   141.04 (±49.07)   
           sphere     1                      0.20 (±0.08)   
exp3       rastrigin  1                     13.76 (±4.26)   
           rosenbrock 1                 4,166.22 (±65.99)   
           sphere     1                      5.11 (±0.84)   
exp4       rastrigin  1                     60.26 (±3.41)   
           rosenbrock 1     14,429,554.87 (±1,972,935.14)   
           sphere     1                     57.40 (±4.71)   

method                                      NNnorm_Worst  \
experiment function   freq                                 
exp1       rastrigin  1                    16.11 (±4.86)   
           rosenbrock 1           71,776.33 (±82,255.12)   
           sphere     1                     7.15 (±4.94)   
exp2       rastrigin  1                     1.96 (±0.20)   
           rosenbrock 1                    67.30 (±8.00)   
           sphere     1                     0.11 (±0.01)   
exp3       rastrigin  1                    14.32 (±3.33)   
           rosenbrock 1               3,249.71 (±962.82)   
           sphere     1                     4.73 (±0.77)   
exp4       rastrigin  1                    59.93 (±4.74)   
           rosenbrock 1     9,526,471.55 (±1,478,125.00)   
           sphere     1                    41.35 (±3.53)   

method                                          noNNReval  
experiment function   freq                                 
exp1       rastrigin  1                     21.90 (±0.47)  
           rosenbrock 1             83,986.73 (±3,445.84)  
           sphere     1                     23.37 (±0.61)  
exp2       rastrigin  1                      1.82 (±0.24)  
           rosenbrock 1                  214.89 (±387.12)  
           sphere     1                      0.45 (±0.14)  
exp3       rastrigin  1                     16.69 (±4.30)  
           rosenbrock 1                    78.06 (±48.66)  
           sphere     1                      0.02 (±0.00)  
exp4       rastrigin  1                    174.62 (±2.23)  
           rosenbrock 1     39,779,456.13 (±3,340,372.07)  
           sphere     1                    143.71 (±8.38)

In [22]:
print(final_data.to_csv(float_format='%.2f'))

experiment,function,freq,NNdrop_Worst,NNnorm_Worst,noNNReval
exp1,rastrigin,1,20.96 (±4.40),16.11 (±4.86),21.90 (±0.47)
exp1,rosenbrock,1,"201,409.54 (±165,798.33)","71,776.33 (±82,255.12)","83,986.73 (±3,445.84)"
exp1,sphere,1,10.86 (±7.31),7.15 (±4.94),23.37 (±0.61)
exp2,rastrigin,1,1.90 (±0.22),1.96 (±0.20),1.82 (±0.24)
exp2,rosenbrock,1,141.04 (±49.07),67.30 (±8.00),214.89 (±387.12)
exp2,sphere,1,0.20 (±0.08),0.11 (±0.01),0.45 (±0.14)
exp3,rastrigin,1,13.76 (±4.26),14.32 (±3.33),16.69 (±4.30)
exp3,rosenbrock,1,"4,166.22 (±65.99)","3,249.71 (±962.82)",78.06 (±48.66)
exp3,sphere,1,5.11 (±0.84),4.73 (±0.77),0.02 (±0.00)
exp4,rastrigin,1,60.26 (±3.41),59.93 (±4.74),174.62 (±2.23)
exp4,rosenbrock,1,"14,429,554.87 (±1,972,935.14)","9,526,471.55 (±1,478,125.00)","39,779,456.13 (±3,340,372.07)"
exp4,sphere,1,57.40 (±4.71),41.35 (±3.53),143.71 (±8.38)



In [ ]:
!pwd

In [ ]:
data.to_csv('tmp.csv', float_format='%.2f')

In [ ]:
!rm tmp.csv

In [ ]:
Func_names=['sphere','rastrigin','rosenbrock']
exp_names=['exp1','exp2','exp3','exp4']
frequency=1000
nn_window=5


In [ ]:
#Experiment = Enum('Experiment', 'exp1 exp2 exp3 exp4')
#Method = Enum('Methods', 'noNNRestart noNNReval NNnorm NNdrop')
#Mechname:Random, Worst, Closest
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0
total=[]
#lables = ['noNNReval', 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']
lables = ['noNNReval', 'NNnorm_Worst', 'NNdrop_Worst']

for experiment in exp_names:
    
    for func in Func_names:
       # path = Path(f'../../data/results/{experiment}/{func}')
        path_in = Path(f'../../data/cluster_results/{experiment}/{func}')
        path = path_in / f'freq{frequency}nn_w{nn_window}nn_p{3}'
        noNNReval= pd.read_csv(path/f'noNNReval_{measure_name[index_measure]}.csv').mean(0).values[0]

        #noNNRestart= pd.read_csv(path/'noNNRestart_mof.csv')

       # NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


       # NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        #y=[noNNReval, NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        y=[noNNReval, NNnorm_Worst,NNdrop_Worst]

        total.append(y)
        #print(total)

    #,  index=[Func_names,Func_names]

#df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])
df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock', 'sphere','rastrigin','rosenbrock'])

#pd.options.display.float_format = '{:,.2f}'.format


print(pd.DataFrame.to_csv(df, float_format='%.2f'))


#pd.DataFrame.to_latex(df, multirow=True)    #, float_format="%%.2f" ,, , longtable=True,  bold_rows=True,


In [ ]:
pd.read_csv('tmp.csv')

In [ ]:
df

In [ ]:
#report nn_time
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=4
total=[]
lables = [ 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']

for experiment in exp_names:
    
    for func in Func_names:
        path = Path(f'../../data/results/{experiment}/{func}')

        NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]

        NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        y=[ NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        total.append(y)
        #print(total)


df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])



print(pd.DataFrame.to_csv(df, float_format='%.2f'))
#df
